In [ ]:
!pip install -q langchain langchain_core langchain_groq langgraph gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 8.7 MB/s eta 0:00:00


In [ ]:
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import gradio as gr

# Set up LLM
llm = ChatGroq(
    temperature=0,
    groq_api_key="Enter here",  # Replace with your key
    model_name="llama-3.3-70b-versatile"
)


In [ ]:
class PlannerState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "Conversation history"]
    city: str
    interests: List[str]
    weather: str
    places: List[str]
    itinerary: str


In [ ]:
import requests

def get_weather(city: str) -> str:
    # Get lat/lon from city using geocoding API
    geo_url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1"
    geo_resp = requests.get(geo_url)
    geo_data = geo_resp.json()

    if "results" not in geo_data or not geo_data["results"]:
        return f"Couldn't find weather info for {city}."

    location = geo_data["results"][0]
    lat, lon = location["latitude"], location["longitude"]

    # Fetch weather using weather API
    weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
    weather_resp = requests.get(weather_url)
    weather_data = weather_resp.json()

    current = weather_data.get("current_weather", {})
    if not current:
        return f"Couldn't retrieve weather data for {city}."

    temp = current["temperature"]
    wind = current["windspeed"]
    desc = f"{temp}°C with wind speeds of {wind} m/s"

    return f"The weather in {city} is currently {desc}."

def search_places(city: str, interests: List[str]) -> List[str]:
    sample_places = {
        "museum": f"{city} Art Museum",
        "park": f"{city} Central Park",
        "coffee": f"{city} Roasters Cafe",
        "nature": f"{city} Botanical Gardens"
    }
    matched = []
    for interest in interests:
        for key, place in sample_places.items():
            if key in interest.lower():
                matched.append(place)
    return matched or [f"Explore local attractions in {city}."]


In [ ]:
def generate_itinerary(state: PlannerState) -> PlannerState:
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful travel assistant. The user is visiting {city}. The weather is: {weather}. Their interests include: {interests}. Provide a concise, bulleted day trip itinerary that takes the weather into account."),
        ("human", "Create my day trip itinerary.")
    ])

    messages = prompt.format_messages(
        city=state["city"],
        interests=", ".join(state["interests"]),
        weather=state["weather"]
    )

    response = llm.invoke(messages)
    state["itinerary"] = response.content
    state["messages"].append(AIMessage(content=response.content))
    return state


In [ ]:
def input_handler(state: PlannerState) -> PlannerState:
    return state

def fetch_weather(state: PlannerState) -> PlannerState:
    weather = get_weather(state['city'])
    state['weather'] = weather
    state['messages'].append(AIMessage(content=f"Weather: {weather}"))
    return state

def find_places(state: PlannerState) -> PlannerState:
    places = search_places(state['city'], state['interests'])
    state['places'] = places
    state['messages'].append(AIMessage(content=f"Places of interest: {', '.join(places)}"))
    return state

def generate_itinerary(state: PlannerState) -> PlannerState:
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful travel assistant. The user is visiting {city}. The weather is: {weather}. Their interests include: {interests}. Provide a concise, bulleted day trip itinerary that takes the weather into account."),
        ("human", "Create my day trip itinerary.")
    ])

    messages = prompt.format_messages(
        city=state["city"],
        interests=", ".join(state["interests"]),
        weather=state["weather"]
    )

    response = llm.invoke(messages)
    state["itinerary"] = response.content
    state["messages"].append(AIMessage(content=response.content))
    return state


In [ ]:
graph = StateGraph(PlannerState)
graph.add_node("InputHandler", input_handler)
graph.add_node("FetchWeather", fetch_weather)
graph.add_node("FindPlaces", find_places)
graph.add_node("GenerateItinerary", generate_itinerary)

graph.set_entry_point("InputHandler")
graph.add_edge("InputHandler", "FetchWeather")
graph.add_edge("FetchWeather", "FindPlaces")
graph.add_edge("FindPlaces", "GenerateItinerary")
graph.add_edge("GenerateItinerary", END)

app = graph.compile()


In [ ]:
def travel_planner_agent(city: str, interests: str) -> str:
    initial_state = {
        "messages": [],
        "city": city,
        "interests": [i.strip() for i in interests.split(",")],
        "weather": "",
        "places": [],
        "itinerary": ""
    }
    final_state = app.invoke(initial_state)
    return final_state["itinerary"]

gr.Interface(
    fn=travel_planner_agent,
    inputs=[
        gr.Textbox(label="Enter your city"),
        gr.Textbox(label="Enter interests (comma-separated)")
    ],
    outputs=gr.Textbox(label="Your personalized itinerary"),
    title="AI Travel Agent (LangGraph)",
    description="Plans a custom day trip using reasoning and tool use"
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a518e44354267d4c75.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
